In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import math

In [17]:
import networkx as nx

In [22]:
xls = pd.ExcelFile("C:\\Users\\abhic\\Modelling_in_Operation_Management\\VRPCDforPerishableproductsDataSet.xlsx")

In [30]:
df1 = pd.read_excel(xls, 'a_iter', header = 1)
df2 = pd.read_excel(xls, 'b_iter', header = 1)
df3 = pd.read_excel(xls, 'c_iter', header = 1)
df4 = pd.read_excel(xls, 'Dip_iter', header = 1)
df5 = pd.read_excel(xls, 'fl_iter', header = 1)
df6 = pd.read_excel(xls, 'Wp_iter', header = 1)
df7 = pd.read_excel(xls, 'u_il', header = 1)

In [31]:
df1.head()

,a_iter,(parameter),optimistic customer delivery time window for order i
0,iter,i,Value
1,iter1,0,50
2,iter1,1,132
3,iter1,2,96
4,iter1,3,66


In [32]:
df2.head()

,b_iter,(parameter),pessimestic customer delivery time window for order
0,iter,i,Value
1,iter1,0,60
2,iter1,1,142
3,iter1,2,106
4,iter1,3,76


In [36]:
df3.head()

,c_iter,(parameter),Unnamed: 2,Unnamed: 3
0,iter,i,j,Value
1,iter1,0,1,7
2,iter1,0,2,8
3,iter1,0,3,13
4,iter1,0,4,5


In [37]:
df4.head()

,Dip_iter,(parameter),size of order i in pallet,Unnamed: 3
0,iter,i,p,Value
1,iter1,0,p1,20
2,iter1,0,p2,12
3,iter1,0,p3,20
4,iter1,1,p1,16


In [38]:
df5.head()

,fl_iter,(parameter),freshness life of product type p
0,iter,p,Value
1,iter1,p1,112
2,iter1,p2,67
3,iter1,p3,97
4,iter2,p1,109


In [39]:
df6.head()

,Wp_iter,(parameter),relative weighted importance of the freshness of product type p
0,iter,p,Value
1,iter1,p1,0.141896
2,iter1,p2,0.404695
3,iter1,p3,0.26389
4,iter2,p1,0.942691


In [40]:
df7.head()

,u,(parameter),if order i is brought by inbound vehicle l and 0 otherwise
0,i,l,Value
1,1,1,1
2,2,1,1
3,3,2,1
4,4,2,1


In [41]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)

(201, 3)
(201, 3)
(1761, 4)
(567, 4)
(61, 3)
(61, 3)
(8, 3)


In [2]:
def _get_routing(G, cross_dock):
    def get_path(u):
        path = [u]
        if u == cross_dock:
            return []
        next_nodes = G.edge[u].keys()
        assert len(next_nodes) == 1
        path.extend(get_path(next_nodes[0]))
        return path

    routing = defaultdict(lambda: [[], []])
    for v, data in G.edge[cross_dock].iteritems():
        vehicle = data[utils.VEHICLE]
        route_type = data[utils.NODE_TYPE]
        index = route_type - 1
        path = get_path(v)
        routing[vehicle][index] = path
    return routing.values()

In [3]:
def tabu_search_vrpcd(G, cross_dock, Q, T, load, tolerance=20, L=12, k=2, a=10,
                      diversification_iter=0):
    """
    Implementation of a Tabu Search algorithm for solving VRPCD.
    Algorithm uses a 2.5 opt move to generate neighbor solution at every
    iteration.
    :param G:, graph, a graph with nodes which represents suppliers and customers.
    :param cross_dock: label of node representing cross-dock.
    :param Q: int, maximum capacity for every vehicle.
    :param T: int, maximum route duration.
    :param load: int, constant time of load's preparation.
    :param tolerance: int, optional (default=20)
    Number of consecutive iterations of algorithm loop that cost of best
    solution is not decreased.
    :param L: int, optional (default=12)
    Maximum length of tabu list.
    :param k:, int, optional (default=2)
    Constant representing the significance to use less vehicles.
    :param a: int, optional (default=2)
    Constant representing the significance on finding less frequent edges during
    on the diversification process.
    :param diversification_iter: int, optional (default=None)
    Iterations of diversification process. If ``None`` as many as ``tolerance``
    diversification iterations will be completed.
    :return: G, graph representing the best solution, float, cost of best
    solution.
    """
    # Calculate distance for every pair of nodes and save it in a dictionary.
    dist = defaultdict(dict)
    for u in G:
        for v in G:
            dist[u][v] = math.sqrt((G.node[u][utils.X_AXIS] - G.node[v][utils.X_AXIS]) ** 2 + (
        G.node[u][utils.Y_AXIS] - G.node[v][utils.Y_AXIS]) ** 2.0)

    if G.number_of_edges() == 0:
        vehicle_id = 0
        capacity = {}
        duration = {}
        for u in G:
            if G.node[u][utils.NODE_TYPE] == utils.SUPPLIER:
                capacity[vehicle_id] = [G.node[u][utils.QUANTITY],
                                        G.node[u][utils.QUANTITY]]
                duration[vehicle_id] = 2 * dist[cross_dock][u]\
                                       + 2 * dist[cross_dock][G.node[u][utils.PAIR]]
                vehicle_id += 1

        # Construct an initial-primitive solution.
        G = construct_primitive_solution(G, cross_dock, dist)
        used_vehicles = sum(1 for vehicle in capacity.keys() if
                            capacity[vehicle][0] != 0 and capacity[vehicle][1] != 0)
    frequency = {(u, v): 1 if G.has_edge(u, v) else 0 for u in G for v in G}
    pickup_tour = {i: sum(data['weight'] for u, v, data in G.edges(data=True)
                          if data[utils.NODE_TYPE] == utils.SUPPLIER and data[utils.VEHICLE] == i)
                   for i in duration.keys()}

    cost = sum(duration.values())
    best_cost = float('inf')
    best_sol = None
    tabu_list = []
    max_iter = 0
    diversification_process = False
    diver_iter = 0
    diversification_iter = tolerance if diversification_iter is None \
        else diversification_iter

    # Tabu algorithm's main loop.
    while max_iter < tolerance:

        cost = _apply_move(G, cross_dock, cost, best_cost, dist,
                           tabu_list, capacity, duration, frequency,
                           pickup_tour, load, Q, T, L,
                           used_vehicles, k, a, diversification_process)
        if cost is None:
            # Not any feasible solution found.
            break
        if cost < best_cost:
            max_iter = 0
            best_cost = cost
            best_sol = G.copy()
        else:
            if not diversification_process:
                max_iter += 1
        # Count diversification iterations.
        if diversification_process:
            diver_iter += 1
            if diver_iter == diversification_iter:
                diversification_process = False

        # Activate diversification process.
        if max_iter == tolerance and diver_iter == 0:
            if not diversification_process and diversification_iter != 0:
                max_iter = 0
            diversification_process = True

    return best_sol, best_cost, _get_routing(best_sol, cross_dock)

In [4]:
def clarke_wright(G, cross_dock, dist, duration, capacity, Q, T):
    """
    Implementation of Clarke-Wright algorithm to construct an initial solution
    for Tabu Search local search algorithm.
    :param G:, graph, a graph representing the initial-primitive solution needed
    for this algorithm.
    :param cross_dock: label of node representing cross-dock.
    :param dist: array of distance between every pair of nodes.
    :param duration, dict, duration of every vehicle's route.
    :param capacity, array of used capacity of every vehicle.
    :param Q: int, maximum capacity for every vehicle.
    :param T: int, maximum route duration.
    :return: G, graph representing the initial solution for Tabu Search.
    """
    savings = {}
    for u in G:
        for v in G:
            if u != cross_dock and v != cross_dock and u != v \
                    and G.node[u][utils.NODE_TYPE] == G.node[v][utils.NODE_TYPE]:
                pair_u = G.node[u][utils.PAIR]
                pair_v = G.node[v][utils.PAIR]
                savings[(u, v)] = dist[cross_dock][u] + dist[cross_dock][v] \
                                  - dist[u][v] + dist[cross_dock][pair_u] \
                                  + dist[cross_dock][pair_v] - dist[pair_u][pair_v]

    import operator

    sorted_savings = sorted(savings.items(), key=operator.itemgetter(1),
                            reverse=True)

    for (u, v), savings in sorted_savings:
        pair_u = G.node[u][utils.PAIR]
        pair_v = G.node[v][utils.PAIR]
        if (G.has_edge(cross_dock, u)) and (G.has_edge(v, cross_dock))\
                and (not G.has_edge(u, v)):
            index, pair_index = (0, 1)\
                if G.node[v][utils.NODE_TYPE] == utils.SUPPLIER else (1, 0)

            # Check if vehicle's capacity is sufficient to service new node.
            if capacity[G.node[v][utils.VEHICLE]][index] + G.node[u][utils.QUANTITY] > Q \
                    and capacity[G.node[pair_v][utils.VEHICLE]][pair_index] + \
                            G.node[u][utils.QUANTITY] > Q:
                continue

            # Initialize object to check the feasibility of solution
            # based on its duration.
            if duration[G.node[v][utils.VEHICLE]] + (
                    dist[pair_u][pair_v] + dist[u][v] - dist[cross_dock][v]) - (
                    dist[cross_dock][pair_v] + dist[cross_dock][u]
                    + dist[cross_dock][pair_u]) > T:
                continue

            if not G.has_edge(u, cross_dock) or not G.has_edge(cross_dock, v) \
                    or not G.has_edge(pair_u, cross_dock) or not G.has_edge(
                    cross_dock, pair_v):
                continue

            # Update capacity of vehicles.
            capacity[G.node[v][utils.VEHICLE]][index] += G.node[u][utils.QUANTITY]
            capacity[G.node[v][utils.VEHICLE]][pair_index] += G.node[u][utils.QUANTITY]
            capacity[G.node[u][utils.VEHICLE]][index] -= G.node[u][utils.QUANTITY]
            capacity[G.node[u][utils.VEHICLE]][pair_index] -= G.node[u][utils.QUANTITY]

            # Update duration of vehicles.
            duration[G.node[v][utils.VEHICLE]] += (
                dist[pair_u][pair_v] + dist[u][v] - dist[cross_dock][v]
                - dist[cross_dock][pair_v] + dist[cross_dock][u] + dist[cross_dock][pair_u])
            duration[G.node[u][utils.VEHICLE]] -= (
                dist[u][cross_dock] + dist[cross_dock][u]) + (
                dist[pair_u][cross_dock] + dist[cross_dock][pair_u])

            # Update solution.
            G.node[u][utils.VEHICLE] = G.node[v][utils.VEHICLE]
            G.node[pair_u][utils.VEHICLE] = G.node[v][utils.VEHICLE]
            G.remove_edge(u, cross_dock)
            G.remove_edge(cross_dock, v)
            G.edge[cross_dock][u][utils.VEHICLE] = G.node[v][utils.VEHICLE]
            G.edge[cross_dock][pair_u][utils.VEHICLE] = G.node[v][utils.VEHICLE]
            G.add_edge(u, v, weight=dist[u][v], type=G.node[v][utils.NODE_TYPE],
                       vehicle=G.node[v][utils.VEHICLE])
            G.remove_edge(pair_u, cross_dock)
            G.remove_edge(cross_dock, pair_v)
            G.add_edge(pair_u, pair_v, weight=dist[pair_u][pair_v],
                       type=G.node[pair_v][utils.NODE_TYPE], vehicle=G.node[v][utils.VEHICLE])
    return G

In [5]:
def construct_primitive_solution(G, cross_dock, dist):
    """
    Constructs a fist solution for Clarke-Wright algorithm.
    This solution uses for every pair of nodes (supplier-customer) a different
    vehicle to serve them.
    :param G: graph, a graph with nodes which represents suppliers and customers.
    :param cross_dock: label of node which represents cross-dock.
    :param pair: string, node data key corresponding to the label of node's pair.
    :param dist: array of distance between every pair of nodes.
    """
    vehicle_id = 0
    for u in G:
        if G.node[u][utils.NODE_TYPE] == utils.SUPPLIER:
            pair_node = G.node[u][utils.PAIR]
            G.node[u][utils.VEHICLE] = vehicle_id
            G.node[pair_node][utils.VEHICLE] = vehicle_id
            G.add_edge(cross_dock, u, weight=dist[cross_dock][u],
                       type=G.node[u][utils.NODE_TYPE], vehicle=vehicle_id)
            G.add_edge(u, cross_dock, weight=dist[u][cross_dock],
                       type=G.node[u][utils.NODE_TYPE], vehicle=vehicle_id)
            G.add_edge(cross_dock, pair_node,
                       weight=dist[cross_dock][pair_node],
                       type=G.node[pair_node][utils.NODE_TYPE], vehicle=vehicle_id)
            G.add_edge(pair_node, cross_dock,
                       weight=dist[pair_node][cross_dock],
                       type=G.node[pair_node][utils.NODE_TYPE], vehicle=vehicle_id)
            vehicle_id += 1
    return G

In [6]:
def _apply_move(G, cross_dock, cost, min_cost, dist, tabu_list, capacity,
                duration, frequency, pickup, load, Q, T, L, used,
                k, a, diversification):
    """
    Apply a move to a current solution to generate neighbor solutions.
    Use a 1-0 exchange move. It moves an element of solution in a new position.
    For example if we apply 1-0 exchange in the solution  = [3, 2, 1, 4, 3]
    we can get the following the transfer of 4 element to the second position.
    A' = [3, 4, 2, 1, 3].
    Then keep the best feasible neighbor solution to be current solution to the
    next iteration of algorithm's loop.
    :param G, graph which represents current solution.
    :param min_cost, float, overall minimum cost.
    :param dist, array of distance between any pair of nodes.
    :param tabu_list, list of characteristics which are tabu in order to be
    excluded.
    :param capacity, array of used capacity of every vehicle.
    :param duration, dict, duration of every vehicle's route.
    :param frequency: dict, frequency for every possible edge.
    :param pickup: dict, pickup tour's duration for every vehicle.
    :param load, int, constant time of load's preparation.
    :param Q, int, maximum vehicle capacity.
    :param T, int maximum route duration for every vehicle.
    :param L, int, maximum length of tabu list.
    loads/unloads every vehicle does at the cross-dock.
    :param used: int, number of used vehicles on current solution.
    :param diversification: bool, If True search for solution which been rarely
    on previous solutions.
    :param k: int, constant representing the significance to use less vehicles.
    :param a: int, constant representing the significance on finding less
    frequent edges during on the diversification process.
    :return Cost of best neighbor solution (which is now current solution for
    next iteration of the algorithm.
    """
    min_neighbor_cost = float('inf')
    min_neighbor_move = ()
    for n, node_data in G.nodes(data=True):

        # Ignore node which represents cross-dock.
        if n == cross_dock:
            continue

        for u, v, data in G.edges(data=True):
            if data[utils.NODE_TYPE] != node_data[utils.NODE_TYPE]:
                continue

            if n == u or n == v:
                continue

            index = 0 if data[utils.NODE_TYPE] == utils.SUPPLIER else 1

            # Check if vehicle's capacity is sufficient to service new node.
            if capacity[data[utils.VEHICLE]][index] + node_data[utils.QUANTITY] > Q:
                continue

            # Initialize object to check the feasibility of solution
            # based on its duration.
            checker = TimeChecker(G, cross_dock, data[utils.VEHICLE],
                                  node_data[utils.VEHICLE],
                                  G.node[node_data[utils.PAIR]][utils.VEHICLE], n,
                                  duration)

            # Check if vehicle's total duration does not surpress T.
            if not checker.is_feasible(dist, u, v, node_data[utils.QUANTITY],
                                       load, T, pickup):
                continue
            withdrawn = True if sum(capacity[node_data[utils.VEHICLE]]) \
                                - node_data[utils.QUANTITY] == 0 else False
            used_vehicles = used - 1 if withdrawn else used

            # Set extra cost for edges which have been previously appeared
            #  in many solution.
            frq_cost = 0 if not diversification else a * (frequency[(u, n)] + frequency[n, v]) ** 2

            # Calculate overall cost of neighbor's solution.
            neighbor_cost = (checker.new_vehicle_dur - duration[
                checker.vehicle]) + (
                checker.previous_vehicle_dur - duration[
                    checker.previous_vehicle]) + (
                                checker.pair_vehicle_dur - duration[
                                    checker.pair_vehicle]) + used_vehicles ** k + frq_cost

            if neighbor_cost < min_neighbor_cost:
                # Check if best (so far neighbor solution) is in the tabu list.
                if any((u, (n, v)) == tabu[0] or (v, (u, n)) == tabu[1] or (
                        n, (u, v)) == tabu[2]
                       for tabu in tabu_list) and (
                                neighbor_cost + cost - frq_cost >= min_cost):
                    continue

                min_neighbor_move = (n, u, v, node_data[utils.NODE_TYPE],
                                     data[utils.VEHICLE])
                min_neighbor_cost = neighbor_cost
                best = index
                vehicles = checker.vehicle, checker.previous_vehicle, \
                           checker.pair_vehicle
                consolidations = checker.consolidation_vehicle, \
                                 checker.consolidation_previous, \
                                 checker.consolidation_pair
                new_durations = checker.new_vehicle_dur, \
                                checker.previous_vehicle_dur, \
                                checker.pair_vehicle_dur
                withdrawn_vehicle = withdrawn

    if min_neighbor_move == ():
        return None

    # Adapt graph based on the current solution which is the best neighbor
    # solution.
    update_edges(G, min_neighbor_move, dist, frequency, pickup)
    update_node_data(G, cross_dock, min_neighbor_move[0], vehicles,
                     new_durations, consolidations, capacity, duration, best)
    update_tabu_list(min_neighbor_move, tabu_list, L)
    used -= 1 if withdrawn_vehicle else 0
    return sum(duration.values())

In [7]:
def update_tabu_list(neighbor_move, tabu_list, L):
    """
    Adds characteristics of best neighbor solution to the tabu list.
    :param neighbor_move: tuple, characteristics of neighbor solution.
    :param tabu_list: list, current tabu list
    :param L: int, maximum length of tabu list.
    """
    n, u, v, _, _ = neighbor_move
    if len(tabu_list) >= L:
        tabu_list.pop(0)
    tabu_list.append(((u, (n, v)), (v, (u, n)), (n, (v, u))))

In [8]:
def update_node_data(G, cross_dock, n, vehicles, new_durations,
                     new_consolidations, capacity, duration, best):
    """
    Updates data of node to be compatible with the best neighbor solution
    (current solution for next iteration).
    :param G: graph, current solution of VRPCD.
    :param n: int, node's label which was moved.
    :param vehicles: tuple, (current vehicle which serves moved node,
    previous vehicle which served moved node, vehicle which serves pair of moved
    node).
    :param new_durations: tuple, new duration's times for current, previous and
    pair vehicles.
    :param new_consolidations: tuple, number of unloads/reloads for current,
    previous, and par vehicles.
    :param capacity: dict, used capacity for every vehicle.
    :param duration: dict, duration of every vehicle's route.
    :param best: int, index which defines if there was an addition/deletion to
    the number of unloads or loads which is defined by the best neighbor
    solution.
    """
    if G.node[n][utils.VEHICLE] != vehicles[0]:
        index = 0 if G.node[n][utils.NODE_TYPE] == utils.SUPPLIER else 1
        capacity[G.node[n][utils.VEHICLE]][index] -= G.node[n][utils.QUANTITY]
        G.node[n][utils.VEHICLE] = vehicles[0]
        capacity[vehicles[0]][index] += G.node[n][utils.QUANTITY]
    for i in range(3):
        duration[vehicles[i]] = new_durations[i]
        pair_index = 0 if best == 1 else 1
        G.node[cross_dock][utils.CONSOLIDATION][vehicles[0]][best] = new_consolidations[0]
        G.node[cross_dock][utils.CONSOLIDATION][vehicles[1]][best] = new_consolidations[1]
        G.node[cross_dock][utils.CONSOLIDATION][vehicles[2]][pair_index] = new_consolidations[2]

In [9]:
def update_edges(G, best_neighbor, dist, frequency, pickup):
    """
    Updates edges (edges removal/addition) to be compatible with the best
     neighbor solution (current solution for next iteration).
    :param G: graph, current solution of VRPCD.
    :param neighbor_move: tuple, characteristics of neighbor solution.
    :param dist: array of distance between any pair of nodes.
    :param frequency: dict, frequency for every possible edge.
    :param pickup: dict, pickup tour's duration for every vehicle.
    """
    n, u, v, node_type, vehicle = best_neighbor
    pred = G.predecessors(n)[0]
    succ = G.successors(n)[0]
    G.remove_edge(u, v)
    G.remove_edge(n, succ)
    G.remove_edge(pred, n)
    if pred != succ:
        frequency[(pred, succ)] += 1
        G.add_edge(pred, succ, weight=dist[pred][succ], type=node_type,
                   vehicle=G.node[n][utils.VEHICLE])
    G.add_edge(u, n, weight=dist[u][n], type=node_type, vehicle=vehicle)
    G.add_edge(n, v, weight=dist[n][v], type=node_type, vehicle=vehicle)
    frequency[(u, n)] += 1
    frequency[(n, v)] += 1
    if node_type == utils.SUPPLIER:
        if G.node[n][utils.VEHICLE] == vehicle:
            pickup[vehicle] += dist[u][n] + dist[n][v] + (
                 - dist[u][v] - dist[n][succ] - dist[pred][n] + dist[pred][succ])
        else:
            pickup[vehicle] += dist[u][n] + dist[n][v] - dist[u][v]
            pickup[G.node[n][utils.VEHICLE]] += - dist[n][succ] - dist[pred][n] + dist[pred][succ]


In [10]:
class TimeChecker:
    """
    This class is a checker of feasibility of one solution based on the duration
    of every vehcile
    """

    def __init__(self, G, cross_dock, vehicle, previous_vehicle, pair_vehicle,
                 node, duration):
        self.G = G
        self.vehicle = vehicle
        self.node = node
        self.duration = duration
        self.pair_vehicle = pair_vehicle
        self.previous_vehicle = previous_vehicle
        self.new_vehicle_dur = 0
        self.previous_vehicle_dur = 0
        self.pair_vehicle_dur = 0
        self.type = G.node[self.node][utils.NODE_TYPE]
        self.consolidation = self.G.node[cross_dock][utils.CONSOLIDATION]

    def is_feasible(self, dist, u, v, quantity, load, T, pickup):
        """
        Checks if solution is feasible by checking that no vehicle's duration
         surpresses the maximum route's duration.
        :param dist, array of distance between any pair of nodes.
        :param u, label of source node of edge that node is moving.
        :param v, label of target node of edge that node is moving.
        :param quantity, int, quantity that has to be picked up from a supplier
         or delivered to a customer.
        :param load, int, constant time of load's preparation.
        :param T, int, maximum route duration for every vehicle.
        :return True if solution is feasible; False otherwise.
        """
        pred = self.G.predecessors(self.node)[0]
        succ = self.G.successors(self.node)[0]
        self.calculate_new_durations(dist, u, v, pred, succ,
                                     self.calculate_consolidation(load,
                                                                  quantity))
        if not self.check_pickup_tours(u, v, pred, succ, pickup, dist):
            return False
        if self.new_vehicle_dur > T:
            return False
        if self.previous_vehicle_dur > T:
            return False
        if self.pair_vehicle_dur > T:
            return False
        return True

    def calculate_new_durations(self, dist, u, v, pred, succ, cd_time):
        """
        This method sets new total duration of every vehicle's trip (vehicle
        which now serves node, vehicle which previously served node, vehicle
        which serves pair of node).
        :param dist, array, distance from every pair of nodes.
        :param u, label of source node of edge that node is moving.
        :param v, label of target node of edge that node is moving.
        :param pred: label of predecessor node of the node is moving.
        :param succ: label of successor node of the node is moving.
        :param quantity, int, quantity that has to be picked up from a supplier
         or delivered to a customer.
        :param cd_time Time spending at cross-dock for every vehicle.
        """
        v_cd, previous_cd, pair_cd = cd_time
        self.new_vehicle_dur = self.duration[self.vehicle] + dist[u][self.node] \
                               + dist[self.node][v] - dist[u][v] + v_cd
        self.previous_vehicle_dur = self.duration[self.previous_vehicle] \
                                    + dist[pred][succ] - dist[pred][self.node] \
                                    - dist[self.node][succ] + previous_cd
        self.pair_vehicle_dur = self.duration[self.pair_vehicle] + pair_cd
        if self.vehicle == self.previous_vehicle and self.vehicle != self.pair_vehicle:
            self.new_vehicle_dur = self.duration[self.vehicle]\
                                   + dist[u][self.node] + dist[self.node][v]\
                                   - dist[u][v] + v_cd - dist[pred][self.node]\
                                   - dist[self.node][succ] + dist[pred][succ]
            self.previous_vehicle_dur = self.new_vehicle_dur
        elif self.previous_vehicle == self.pair_vehicle \
                and self.previous_vehicle != self.vehicle:
            self.pair_vehicle_dur = self.previous_vehicle_dur
        elif self.vehicle == self.pair_vehicle \
                and self.vehicle != self.previous_vehicle:
            self.pair_vehicle_dur = self.new_vehicle_dur
        elif self.vehicle == self.pair_vehicle == self.previous_vehicle:
            self.pair_vehicle_dur = self.new_vehicle_dur
            self.previous_vehicle_dur = self.new_vehicle_dur

    def calculate_consolidation(self, load, quantity):
        """
        This method calculates new times spending at cross-dock for
        every vehicle (vehicle which now serves node, vehicle which previously
        served node, vehicle which serves pair of node).
        :param load, int, constant time of load's preparation.
        :param quantity, int, quantity that has to be picked up from a supplier
         or delivered to a customer.
        :return tuple, new times spending at cross-dock for every
        vehicle.
        """
        self.init_cd_consolidations()
        (cd_vehicle, n_cd_vehicle), (cd_previous, n_cd_previous), (cd_pair, n_cd_pair)\
            = self.init_cd_diff_times(load, quantity)
        vehicle_at_cd = 0
        pair_vehicle_at_cd = 0
        previous_vehicle_at_cd = 0
        if self.previous_vehicle != self.vehicle:
            if self.previous_vehicle == self.pair_vehicle:
                previous_vehicle_at_cd = cd_pair
                self.consolidation_pair += 1
            else:
                self.consolidation_previous -= 1
                previous_vehicle_at_cd = n_cd_previous
        if self.vehicle == self.pair_vehicle:
            if self.vehicle != self.previous_vehicle:
                self.consolidation_pair -= 1
                vehicle_at_cd = n_cd_pair
        else:
            if self.vehicle != self.previous_vehicle:
                self.consolidation_vehicle += 1
                vehicle_at_cd = cd_vehicle
            if self.pair_vehicle == self.previous_vehicle:
                pair_vehicle_at_cd = cd_pair
        return vehicle_at_cd, previous_vehicle_at_cd, pair_vehicle_at_cd

    def init_cd_consolidations(self):
        """Initialize number of loads/unloads at the cross-dock for every
        vehicle."""
        index, pair_index = (0, 1) if self.type == utils.SUPPLIER else (1, 0)
        self.consolidation_vehicle = self.consolidation[self.vehicle][index]
        self.consolidation_pair = self.consolidation[self.pair_vehicle][
            pair_index]
        self.consolidation_previous = self.consolidation[self.previous_vehicle][
            index]

    def init_cd_diff_times(self, load, quantity):
        """
        This method calculates the variation of time spending at cross-dock for
        every vehicle (vehicle which now serves node, vehicle which previously
        served node, vehicle which serves pair of node).
        :param load, int, constant time of load's preparation.
        :param quantity, int, quantity that has to be picked up from a supplier
         or delivered to a customer.
        :return tuple, variation of time spending at cross-dock for every
        vehicle.
        """
        diff1 = load + quantity
        diff2 = quantity
        cd_vehicle = diff1 if self.consolidation_vehicle == 0 else diff2
        negative_cd_vehicle = -diff1 if self.consolidation_vehicle == 1 else -diff2
        cd_previous = diff1 if self.consolidation_previous == 0 else diff2
        negative_cd_previous = -diff1 if self.consolidation_previous == 1 else -diff2
        cd_pair = diff1 if self.consolidation_pair == 0 else diff2
        negative_cd_pair = -diff1 if self.consolidation_pair == 1 else - diff2
        return (cd_vehicle, negative_cd_vehicle), \
               (cd_previous, negative_cd_previous), (cd_pair, negative_cd_pair)

    def check_pickup_tours(self, u, v, pred, succ, pickup_tour, dist):
        """
        This method checks if the pick up tour's duration of vehicle which
        unloads items is great than the pick up tour's duration of the vehicle
        which is going to load this item.
        :param u, label of source node of edge that node is moving.
        :param v, label of target node of edge that node is moving.
        :param pred: label of predecessor node of the node is moving.
        :param succ: label of successor node of the node is moving.
        :param pickup_tour: dict, duration of pick up tours for every vehicle.
        :param dist: :param dist, array, distance from every pair of nodes.
        :return: Returns True if solution is feasible based on the pick up tours
        of vehicle; False otherwise.
        """
        variation_pair = dist[pred][succ] - dist[pred][self.node] - dist[self.node][succ]
        if self.previous_vehicle != self.vehicle:
            variation_vehicle = dist[u][self.node] + dist[self.node][v] - dist[u][v]
        else:
            variation_vehicle = dist[u][self.node] + dist[self.node][v]\
                                - dist[u][v] - dist[pred][self.node]\
                                - dist[self.node][succ] + dist[pred][succ]
        if self.vehicle != self.pair_vehicle:
            pair_pick = pickup_tour[self.previous_vehicle] + variation_pair\
                if self.pair_vehicle == self.previous_vehicle and self.type == utils.SUPPLIER\
                else pickup_tour[self.pair_vehicle]
            pick = pickup_tour[self.vehicle] + variation_vehicle if self.type == utils.SUPPLIER\
                else pickup_tour[self.vehicle]
            if self.type == utils.SUPPLIER and 0 < pair_pick < pick:
                return False
            if self.type == utils.CUSTOMER and 0 < pick < pair_pick:
                return False
        return True